## Simple Chain

In [1]:
from dotenv import load_dotenv
import os

env_path = "../OllamaSetup/.env"  # Relative path from PromptTemplates
load_dotenv(env_path)

# Check if variables are loaded
print(os.getenv("LANGCHAIN_ENDPOINT")) 

https://api.smith.langchain.com


In [2]:
from langchain_ollama import ChatOllama
base_url="http://localhost:11434"
#model = 'llama3.2:1b'
# model='sheldon'
model = 'llama3.2:3b'


llm = ChatOllama(
    base_url=base_url,
    model = model)

In [3]:
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate,
                                        PromptTemplate
)

from langchain_core.output_parsers import StrOutputParser

In [4]:
template= ChatPromptTemplate.from_template("{prompt}")
chain= template | llm | StrOutputParser()

about= "My name is xyz. I work for abc"
chain.invoke({'prompt':about})

'Nice to meet you, Xyz from ABC! Is there something I can help you with or would you like to chat?'

In [5]:
prompt= "What is my name?"
chain.invoke({'prompt':prompt})

"I don't have any information about your name. I'm a large language model, I don't retain personal data or remember individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. Would you like to tell me your name?"

## Runnable with Message History

In [6]:
import langchain_community
print("LangChain Community Edition installed successfully!")


LangChain Community Edition installed successfully!


In [17]:
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate,
                                        PromptTemplate,
                                        MessagesPlaceholder
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


In [8]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

In [9]:
runnable_with_history = RunnableWithMessageHistory(chain,get_session_history)

In [10]:
user_id= '121'
history=get_session_history(user_id)
history.get_messages()

/var/folders/v_/q_0xfydn1ysbg3dg6_yj25th0000gn/T/ipykernel_9181/4235610081.py:2: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  history=get_session_history(user_id)


[HumanMessage(content='My name is xyz. I work for abc', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It appears that you are trying to send a message using the HumanMessage object.\n\nHere's an example of how you can use it:\n\n```python\nfrom humans import HumanMessage\n\n# Create a new HumanMessage object\nmessage = HumanMessage(content='My name is xyz. I work for abc', additional_kwargs={}, response_metadata={})\n\nprint(message.content)  # Output: My name is xyz. I work for abc\n```\n\nHowever, without more information about the environment or context in which this code is being used, it's difficult to provide a more specific answer.\n\nIf you could provide more details about what you're trying to accomplish with this code, I'd be happy to try and help further.", additional_kwargs={}, response_metadata={})]

In [11]:
history.clear()

In [12]:
about

'My name is xyz. I work for abc'

In [13]:
runnable_with_history.invoke([HumanMessage(content=about)],
                             config={'configurable': {'session_id' : user_id}})

'It appears you\'re providing a human message in the format expected by the Hugging Face Transformers library.\n\nHere\'s a breakdown of what this code snippet does:\n\n- `[HumanMessage`]: This indicates that the content that follows is a human-generated message.\n- `(content=\'My name is xyz. I work for abc\')`: The `content` parameter contains the actual text message you want to be processed, in this case, "My name is xyz. I work for abc".\n- `(additional_kwargs={})`: This parameter allows you to pass additional keyword arguments that might be required by the model or any downstream processing steps. In this case, it\'s an empty dictionary (`{}`).\n- `(response_metadata={})`: Similar to `additional_kwargs`, this parameter allows you to pass metadata about the response from a human model. It\'s also an empty dictionary in this example.\n\nIn essence, this code snippet is setting up a message for a conversational AI or chatbot that can engage with humans.'

In [14]:
runnable_with_history.invoke([HumanMessage(content="What is my name?")],
                             config={'configurable': {'session_id' : user_id}})

'This appears to be an example of how to create and structure messages in the Hugging Face Transformers library, specifically for use in conversational AI or chatbot applications.\n\nThe code defines two `HumanMessage` instances:\n\n1. The first message is from a human user (`xyz`) and contains the text "My name is xyz. I work for abc." This indicates that the content is a human-generated message.\n2. The second message is also from a human user, but asks "What is my name?".\n\nBoth messages are created using the `HumanMessage` class, which takes in three parameters:\n- `content`: The actual text of the message.\n- `additional_kwargs`: Additional keyword arguments that might be required by the model or downstream processing steps. In this case, both instances use an empty dictionary (`{}`).\n- `response_metadata`: Metadata about the response from a human model. Again, both instances use an empty dictionary (`{}).`\n\nThe structure of these messages is consistent with what\'s expected b

## Message History with Dictionary Like inputs

In [16]:
system = SystemMessagePromptTemplate.from_template("You are helpful assistant.")
human = HumanMessagePromptTemplate.from_template("{input}")

In [18]:
messages =[system, MessagesPlaceholder(variable_name='history'),human]
prompt = ChatPromptTemplate(messages=messages)
chain= prompt | llm | StrOutputParser()
runnable_with_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key='input',history_messages_key='history')



In [20]:
def chat_with_llm(session_id,input):
    output = runnable_with_history.invoke(
        {'input': input},
        config={'configurable':{'session_id': session_id}}
    )
    return output

In [21]:
user_id='rr'
chat_with_llm(user_id,about)

'Nice to meet you, XYZ from ABC! How can I assist you today? Do you have any questions or need help with something specific?'

In [22]:
chat_with_llm(user_id,"What is my name")

"Your name is XYZ. However, I should note that you didn't give me your actual first and last names earlier. Is there a particular reason why you'd like to share them now?"